<a href="https://colab.research.google.com/github/eunseojeon/AI_Coding_for_Autonomous_Driving_Class/blob/main/0725_%EA%B0%9D%EC%B2%B4%ED%83%90%EC%A7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import shutil
import os

folder = "/content"
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)  # 파일 또는 심볼릭 링크 삭제
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)  # 폴더(디렉토리) 삭제
    except Exception as e:
        print(f'Failed to delete {file_path}. Reason: {e}')

#위 코드는 아래 코드들에서 오류났을 때, 새로운 파일이 생기니까 그 파일을 다 삭제해주는 코드! (리셋느낌)

In [ ]:
# 유튜브 영상을 다운받기 위해서 yt-dlp 설치
!pip install yt-dlp
!pip install ultralytics

from ultralytics import YOLO
import yt_dlp
import tempfile

In [ ]:
# 필수 라이브러리 설치 (최신 yt-dlp)
!pip install -U yt-dlp

# ✅ 유튜브 주소 입력
YOUTUBE_URL = "https://youtu.be/cgdQIgeQE-Y?si=ZtlztX-H3z4taU13"

# 저장 경로 설정
import os
os.makedirs("videos", exist_ok=True)
video_path = "videos/input.mp4"

# yt-dlp 다운로드 옵션
import yt_dlp

ydl_opts = {
    "outtmpl": video_path,                # 저장될 경로
    "format": "best[ext=mp4]/best",       # mp4 확장자 우선
    "noplaylist": True,
    "quiet": False,
    "restrictfilenames": True,
    "retries": 3,
}

# 다운로드 실행
try:
    print(f"⬇️ 유튜브 영상 다운로드 중...\n🎥 {YOUTUBE_URL}")
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([YOUTUBE_URL])
    print(f"✅ 다운로드 완료: {video_path}")
except Exception as e:
    raise RuntimeError(f"❌ 다운로드 실패: {e}")


⬇️ 유튜브 영상 다운로드 중...
🎥 https://youtu.be/cgdQIgeQE-Y?si=ZtlztX-H3z4taU13
[youtube] Extracting URL: https://youtu.be/cgdQIgeQE-Y?si=ZtlztX-H3z4taU13
[youtube] cgdQIgeQE-Y: Downloading webpage
[youtube] cgdQIgeQE-Y: Downloading tv client config
[youtube] cgdQIgeQE-Y: Downloading tv player API JSON
[youtube] cgdQIgeQE-Y: Downloading ios player API JSON
[youtube] cgdQIgeQE-Y: Downloading m3u8 information
[info] cgdQIgeQE-Y: Downloading 1 format(s): 18
[download] Destination: videos/input.mp4
[download] 100% of    3.55MiB in 00:00:00 at 6.86MiB/s   
✅ 다운로드 완료: videos/input.mp4


In [ ]:
from ultralytics import YOLO
import os, glob
import cv2
import time

# 1. 모델 로드
MODEL_WEIGHTS = "yolo11n.pt"   # 또는 "yolov8n.pt"
model = YOLO(MODEL_WEIGHTS)

# 2. 비디오 경로 정하기
video_path = "/content/videos/input.mp4"  # 다운로드 된 영상 경로

# 3. 원본 영상 FPS, 총 프레임, 길이 측정
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise RuntimeError(f"5 영상 열기 실패: {video_path}")

fps_original = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
duration = frame_count / fps_original if fps_original else 0

print("📊 원본 영상 정보")
print(f" - FPS: {fps_original:.2f}")
print(f" - 총 프레임 수: {frame_count}")
print(f" - 길이: {duration:.2f}초")

cap.release()

# 🧠 4. traffic light 클래스 인덱스 추출
names = model.names
if isinstance(names, dict):
    tl_idx = [k for k, v in names.items() if v == 'traffic light'][0]
else:
    tl_idx = names.index('traffic light')

# ⚙️ 5. YOLO 추론 수행 (시간 측정 포함)
save_dir = "runs/detect/model"

start_time = time.time()
results = model.predict(
    source=video_path,
    conf=0.3,
    #classes=[tl_idx], 이건 특정 클래스만 지정하고 싶을 때 쓰는 거
    classes=[0, 1, 2, 7, 9, 11],  # ← 원하는 클래스 번호만 나열!
    save=True,
    project="runs/detect",
    name="all_objects",
    vid_stride=1  # 모든 프레임 사용
)
end_time = time.time()

# 📁 6. 결과 비디오 경로 탐색
out_videos = glob.glob(os.path.join(save_dir, "*.mp4"))
print("\n📁 결과 비디오:", out_videos)

# ⏱️ 7. YOLO 추론 성능 측정
elapsed = end_time - start_time
inferred_frames = len(results)
fps_yolo = inferred_frames / elapsed if elapsed > 0 else 0

print("\n⚙️ YOLO 추론 처리 정보")
print(f" - 전체 처리 시간: {elapsed:.2f}초")
print(f" - 처리된 프레임 수: {inferred_frames}")
print(f" - 평균 추론 FPS: {fps_yolo:.2f} frames/sec")

100%|██████████| 5.35M/5.35M [00:00<00:00, 103MB/s]


📊 원본 영상 정보
 - FPS: 30.00
 - 총 프레임 수: 1200
 - 길이: 40.00초

WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/1200) /content/videos/input.mp4: 384x640 1 car, 1 truck, 196.6ms
video 1/1 (frame 2/1200) /content/videos/input.mp4: 384x640 2 cars, 1 truck, 130.4ms
video 1/1 (frame 3/1200) /content/videos/input.mp4: 384x640 2 cars, 1 truck, 129.4ms
video 1/1 (frame 4/1200) /content/videos/input.mp4: 384x640 2 cars, 1 truck, 145.1ms
video 1/1 (frame 5/1200) /content/videos/input.mp4: 384x640 3

In [ ]:
print("모델 클래스 목록:", model.names)

모델 클래스 목록: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 